In [1]:
import pandas as pd 
import numpy as np
import os
import sys 
from pathlib import Path
import matplotlib.pyplot as plt

import tsforge as tsf

import warnings

* fetch metadata

In [2]:
# Set working directory to project root
ROOT_DIR = Path("../..").resolve()
os.chdir(ROOT_DIR)
sys.path.insert(0, str(ROOT_DIR))


warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8-whitegrid")

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

OUTPUT_DIR = DATA_DIR / "output"
OUTPUT_DIR.mkdir(exist_ok=True)

data = tsf.load_m5(data_dir=DATA_DIR, create_unique_id=True, verbose=False,include_hierarchy=True)


In [3]:
meta_data = data.select_dtypes(include=['category','object']).drop_duplicates(subset='unique_id').reset_index(drop=True)

In [4]:
df = pd.read_parquet(
    "/Users/jackrodenberg/Desktop/real-world-forecasting-foundations/notebooks/module_02_baselines/statsforecast_backtest.parquet",
)

* sku level metrics computation using utilsforecast, across all skus and timesteps

In [5]:
# model_cols = [col for col in df.columns if col not in ['unique_id','ds','y','cutoff']]

# metric_df = evaluate(
#     df = df,
#     metrics=[rmse,mae,bias,nd],
#     id_col='unique_id',
#     time_col='ds',
#     target_col='y',
#     models=model_cols
# )

# # inspect mean of the metrics across all skus and all timesteps
# metric_df.groupby("metric")[model_cols].mean()

* make bespoke metrics out of the baseline dataframe, here we can be more creative with custom error measures

In [6]:
e_df = df.melt(
    id_vars=["unique_id", "ds", "cutoff", "y"], var_name="model", value_name="y_pred"
).assign(
    error=lambda x: x["y"] - x["y_pred"], abs_error=lambda x: np.abs(x["error"]), sq_error = lambda x: np.square(x['error'])
)  # add granular error/abs error columns.. these will be used for bespoke metrics...

e_df = e_df.assign(
    timestep=e_df.groupby(["unique_id", "model", "cutoff"]).cumcount().add(1),
    horizon_group=lambda x: np.where(x["timestep"] > 4, "5-13", "1-4"),
)

e_df.head()


,unique_id,ds,cutoff,y,model,y_pred,error,abs_error,sq_error,timestep,horizon_group
0,FOODS_1_001_CA_1,2015-07-04,2015-06-27,2.0,Naive,2.0,0.0,0.0,0.0,1,1-4
1,FOODS_1_001_CA_1,2015-07-11,2015-06-27,2.0,Naive,2.0,0.0,0.0,0.0,2,1-4
2,FOODS_1_001_CA_1,2015-07-18,2015-06-27,7.0,Naive,2.0,5.0,5.0,25.0,3,1-4
3,FOODS_1_001_CA_1,2015-07-25,2015-06-27,4.0,Naive,2.0,2.0,2.0,4.0,4,1-4
4,FOODS_1_001_CA_1,2015-08-01,2015-06-27,2.0,Naive,2.0,0.0,0.0,0.0,5,5-13


In [ ]:
# ============================================================================
# STEP 3: Aggregate Errors by Model/Horizon/Series/Cutoff
# ============================================================================

groupby_keys = ["model", "horizon_group", "unique_id", "cutoff"]

base_metric_specs = [
    ("mae", "abs_error", "mean"),
    ("sum_ae", "abs_error", "sum"),
    ("sum_demand", "y", "sum"),
    ("bias", "error", "mean"),
    ("mse", "sq_error", "mean"),
    ("stability", "mean_abs_jitter", "sum"),
]

sku_metric_specs = [
    ("sku_mae", "sku_abs_error", "mean"),
    ("sku_sum_ae", "sku_abs_error", "sum"),
    ("sku_bias", "sku_error", "mean"),
    ("sku_mse", "sku_sq_error", "mean"),
]

agg_dict = {
    name: (column, func)
    for name, column, func in base_metric_specs + sku_metric_specs
}

aggregated_errors = (
    e_df.groupby(groupby_keys, sort=False, observed=True, as_index=False)
    .agg(**agg_dict)
)

print("\u2713 Aggregated base- and SKU-level errors")
print(f"  Number of unique combinations: {len(aggregated_errors):,}")
print(f"  Groupby keys: {groupby_keys}")

In [8]:
# Define COV function... 
def coefficient_of_variation(values):
    """Calculate coefficient of variation (CV = std/mean).

    Measures relative variability. Lower CV = more stable forecasts.
    """
    return values.std() / np.maximum(values.mean(), 1e-8)


In [9]:
# ============================================================================
# STEP 1: Optimize DataFrame for Groupby Operations
# ============================================================================
# Converting to category dtype speeds up groupby by ~2-3x

categorical_columns = ["unique_id", "cutoff", "model", "horizon_group"]
e_df[categorical_columns] = e_df[categorical_columns].astype("category")

print(f"✓ Converted {len(categorical_columns)} columns to category dtype")
print(f"  DataFrame shape: {e_df.shape}")


✓ Converted 4 columns to category dtype
  DataFrame shape: (9512880, 16)


* compute jitter as forecast change for the same timestep (i.e., how much the forecast changes from one time step to the next for the same forecast horizon)

* talking point for jitter computation: our current CV config doesn't measure the change for same FTD over FCDs as we have non-overlapping folds.. 

In [ ]:
jitter_df

level_3,unique_id,timestep,model,sku_pred,y_pred
0,FOODS_1_001_CA_1,1,CrostonOptimized,70.461166,0.409770
1,FOODS_1_001_CA_1,1,HW52,70.461166,2.722255
2,FOODS_1_001_CA_1,1,MA4,70.461166,1.416667
3,FOODS_1_001_CA_1,1,Naive,70.461166,2.000000
4,FOODS_1_001_CA_1,1,SN52,70.461166,4.000000
...,...,...,...,...,...
2378215,HOUSEHOLD_2_516_WI_3,13,HW52,5.575422,0.411594
2378216,HOUSEHOLD_2_516_WI_3,13,MA4,5.575422,0.500000
2378217,HOUSEHOLD_2_516_WI_3,13,Naive,5.575422,1.333333
2378218,HOUSEHOLD_2_516_WI_3,13,SN52,5.575422,1.000000


In [18]:
# jitter computation as mean abs jitter 
jitter_df = (
    e_df.pivot(
        index=['unique_id','timestep','model'],
        columns='cutoff',
        values=['y_pred','sku_pred']
    )
    .stack(level=0) # stack index to get jitter for sku-store and sku level
    .diff(axis=1)
    .abs()
    .mean(axis=1)
    .reset_index(name='mean_abs_jitter')
)

jitter_df['level_3'] = jitter_df['level_3'].str.replace("_pred","") + '_jitter'

# pivot jitter df 
jitter_df = jitter_df.pivot(
    index=['unique_id','timestep','model'],
    columns='level_3',
    values='mean_abs_jitter'
).reset_index()

# merge jitter computation on id, timestep and model
e_df = e_df.merge(
    jitter_df,
    on=['unique_id','timestep','model'],
    how='left',
    validate="m:1"
)

In [23]:
e_df

,unique_id,ds,cutoff,y,model,y_pred,error,abs_error,sq_error,timestep,horizon_group,item_id,sku_pred,sku_abs_error,sku_sq_error,sku_error,sku_jitter,y_jitter
0,FOODS_1_001_CA_1,2015-07-04,2015-06-27,2.0,Naive,2.000000,0.000000,0.000000,0.000000,1,1-4,FOODS_1_001,179.347183,48.652817,2367.096680,48.652817,70.461166,2.000000
1,FOODS_1_001_CA_1,2015-07-11,2015-06-27,2.0,Naive,2.000000,0.000000,0.000000,0.000000,2,1-4,FOODS_1_001,180.692017,65.307983,4265.132812,65.307983,87.767159,2.000000
2,FOODS_1_001_CA_1,2015-07-18,2015-06-27,7.0,Naive,2.000000,5.000000,5.000000,25.000000,3,1-4,FOODS_1_001,182.162125,45.837875,2101.110840,45.837875,65.541237,2.000000
3,FOODS_1_001_CA_1,2015-07-25,2015-06-27,4.0,Naive,2.000000,2.000000,2.000000,4.000000,4,1-4,FOODS_1_001,182.956055,2.956053,8.738248,-2.956053,51.375385,2.000000
4,FOODS_1_001_CA_1,2015-08-01,2015-06-27,2.0,Naive,2.000000,0.000000,0.000000,0.000000,5,5-13,FOODS_1_001,181.831528,4.168463,17.376081,4.168463,42.270329,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9512875,HOUSEHOLD_2_516_WI_3,2016-05-28,2016-03-26,0.0,StructuralTheta,0.725285,-0.725285,0.725285,0.526038,9,5-13,HOUSEHOLD_2_516,58.655552,13.344450,178.074341,13.344450,6.860040,0.076095
9512876,HOUSEHOLD_2_516_WI_3,2016-06-04,2016-03-26,0.0,StructuralTheta,0.724110,-0.724110,0.724110,0.524335,10,5-13,HOUSEHOLD_2_516,60.864147,6.864149,47.116535,-6.864149,5.087307,0.075692
9512877,HOUSEHOLD_2_516_WI_3,2016-06-11,2016-03-26,0.0,StructuralTheta,0.722935,-0.722935,0.722935,0.522635,11,5-13,HOUSEHOLD_2_516,72.723564,36.723568,1348.620483,-36.723568,10.932172,0.075288
9512878,HOUSEHOLD_2_516_WI_3,2016-06-18,2016-03-26,3.0,StructuralTheta,0.721760,2.278240,2.278240,5.190378,12,5-13,HOUSEHOLD_2_516,63.811951,2.188049,4.787557,2.188049,10.113582,0.074885


In [22]:
# ============================================================================
# STEP 3: Aggregate Errors by Model/Horizon/Series/Cutoff
# ============================================================================

groupby_keys = ["model", "horizon_group", "unique_id", "cutoff"]

aggregated_errors = e_df.groupby(groupby_keys, sort=False, observed=True, as_index=False).agg(
    mae=("abs_error", "mean"),  # Mean Absolute Error
    sum_ae=("abs_error", "sum"),  # Sum of absolute errors (for wMAPE)
    sum_demand=("y", "sum"),  # Sum of actual demand (for wMAPE)
    bias=("error", "mean"),  # Mean Error (bias)
    mse=("sq_error", "mean"),  # Mean Squared Error (for RMSE)
    store_sku_stability=("y_jitter","sum"), # sum all the jitter values,
    sku_stability=("sku_jitter","sum"), # count all the jitter values
)

print("✓ Aggregated errors")
print(f"  Number of unique combinations: {len(aggregated_errors):,}")
print(f"  Groupby keys: {groupby_keys}")


✓ Aggregated errors
  Number of unique combinations: 1,463,520
  Groupby keys: ['model', 'horizon_group', 'unique_id', 'cutoff']


In [9]:
# ============================================================================
# NORMALIZE MAE BY NAIVE FORECAST PERFORMANCE
# ============================================================================

# Step 1: Extract Naive model's MAE for each unique_id/horizon_group/cutoff
naive_mae = (
    aggregated_errors.query("model == 'Naive'")  # or .loc[aggregated_errors['model'] == 'Naive']
    .set_index(["unique_id", "horizon_group", "cutoff"])["mae"]
    .rename("naive_mae_baseline")
    #.clip(lower=1e-10)
)

print(f"✓ Extracted Naive MAE baseline")
print(f"  Number of Naive benchmarks: {len(naive_mae):,}")


# Step 2: Join back to all models and calculate relative MAE
aggregated_errors = (
    aggregated_errors.set_index(["unique_id", "horizon_group", "cutoff"])
    .assign(
        naive_mae_baseline=naive_mae, mase=lambda df: df["mae"] / df["naive_mae_baseline"]
    )
    .reset_index()
)

print(f"✓ Calculated relative MAE (MAE / Naive MAE)")
print(f"\nRelative MAE by model:")
print(aggregated_errors.groupby("model")["mase"].agg(["mean", "median", "min", "max"]))

✓ Extracted Naive MAE baseline
  Number of Naive benchmarks: 243,920
✓ Calculated relative MAE (MAE / Naive MAE)

Relative MAE by model:
                  mean    median       min  max
model                                          
CrostonOptimized   inf  0.969634  0.000644  inf
HW52               inf  1.115370  0.004659  inf
MA4                inf  1.000000  0.000000  inf
Naive              1.0  1.000000  1.000000  1.0
SN52               inf  1.200000  0.000000  inf
StructuralTheta    inf  1.000032  0.004362  inf


/var/folders/j6/v7r4q64d1zv5qj54955b_k_00000gn/T/ipykernel_18606/335417471.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(aggregated_errors.groupby("model")["mase"].agg(["mean", "median", "min", "max"]))


In [10]:
# ============================================================================
# STEP 4: Calculate Derived Metrics
# ============================================================================

# Calculate RMSE from MSE
aggregated_errors = aggregated_errors.assign(
    rmse=lambda df: np.sqrt(
        df["mse"]
    ),  # Root Mean Squared Error
    # Weighted Mean Absolute Percentage Error
    wMAPE=lambda df: df["sum_ae"] / df["sum_demand"],
)


In [11]:
# ============================================================================
# FINAL RESULT
# ============================================================================

error_table = aggregated_errors

print("\n" + "=" * 70)
print("ERROR TABLE COMPLETE")
print("=" * 70)
print(f"\nShape: {error_table.shape}")
print(f"\nColumns:")
for col in error_table.columns:
    print(f"  - {col}")

print(f"\nMetrics summary:")
print(error_table[["mae", "rmse", "mase", "wMAPE", "bias", "stability"]].describe())



ERROR TABLE COMPLETE

Shape: (1463520, 14)

Columns:
  - unique_id
  - horizon_group
  - cutoff
  - model
  - mae
  - sum_ae
  - sum_demand
  - bias
  - mse
  - stability
  - naive_mae_baseline
  - mase
  - rmse
  - wMAPE

Metrics summary:


/Users/jackrodenberg/Desktop/real-world-forecasting-foundations/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/jackrodenberg/Desktop/real-world-forecasting-foundations/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


                mae          rmse          mase         wMAPE          bias  \
count  1.463520e+06  1.463520e+06  1.434982e+06  1.434360e+06  1.463520e+06   
mean   5.057615e+00  5.895721e+00           inf           inf  4.369787e-01   
std    1.028684e+01  1.151721e+01           NaN           NaN  1.034859e+01   
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00 -5.274183e+02   
25%    1.257363e+00  1.581139e+00  8.586781e-01  4.285714e-01 -1.203492e+00   
50%    2.500000e+00  2.977038e+00  1.000000e+00  6.693226e-01  1.111111e-01   
75%    5.000000e+00  5.873670e+00  1.214753e+00  1.000000e+00  1.750000e+00   
max    8.263656e+02  8.296942e+02           inf           inf  8.263656e+02   

          stability  
count  1.463520e+06  
mean   3.034691e+01  
std    7.298551e+01  
min    0.000000e+00  
25%    5.418661e+00  
50%    1.275775e+01  
75%    2.966234e+01  
max    5.951250e+03  


In [24]:
error_table.loc[(error_table['wMAPE'] == np.inf) | (error_table['mase'] == np.inf)] # when sum of demand is zero, we get a wMAPE of Inf, same to be said for mase often times... 

,unique_id,horizon_group,cutoff,model,mae,sum_ae,sum_demand,bias,mse,stability,naive_mae_baseline,mase,rmse,wMAPE
98,FOODS_1_002_CA_3,1-4,2015-09-26,Naive,1.000000,4.000000,0.0,-1.000000,1.000000,17.333334,1.0,1.000000,1.000000,inf
99,FOODS_1_002_CA_3,5-13,2015-09-26,Naive,1.000000,9.000000,0.0,-1.000000,1.000000,39.000000,1.0,1.000000,1.000000,inf
112,FOODS_1_002_TX_1,1-4,2015-06-27,Naive,1.000000,4.000000,0.0,-1.000000,1.000000,1.333333,1.0,1.000000,1.000000,inf
226,FOODS_1_003_WI_2,1-4,2015-09-26,Naive,2.000000,8.000000,0.0,-2.000000,4.000000,2.666667,2.0,1.000000,2.000000,inf
227,FOODS_1_003_WI_2,5-13,2015-09-26,Naive,2.000000,18.000000,0.0,-2.000000,4.000000,6.000000,2.0,1.000000,2.000000,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463414,HOUSEHOLD_2_515_TX_3,1-4,2016-03-26,StructuralTheta,0.436452,1.745807,0.0,-0.436452,0.190499,0.868468,0.0,inf,0.436462,inf
1463424,HOUSEHOLD_2_515_WI_2,1-4,2015-06-27,StructuralTheta,0.484161,1.936642,0.0,-0.484161,0.234427,0.470373,0.0,inf,0.484176,inf
1463497,HOUSEHOLD_2_516_WI_1,5-13,2015-06-27,StructuralTheta,0.595327,5.357941,0.0,-0.595327,0.354420,1.611150,1.0,0.595327,0.595332,inf
1463506,HOUSEHOLD_2_516_WI_2,1-4,2015-09-26,StructuralTheta,0.238612,0.954449,0.0,-0.238612,0.056936,0.245443,0.0,inf,0.238612,inf


In [23]:
error_table.query("wMAPE != inf and mase != inf").groupby("model")[["mae","rmse","stability","wMAPE","mase"]].agg(['mean','median']).stack()

/var/folders/j6/v7r4q64d1zv5qj54955b_k_00000gn/T/ipykernel_18606/2389047552.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  error_table.query("wMAPE != inf and mase != inf").groupby("model")[["mae","rmse","stability","wMAPE","mase"]].agg(['mean','median']).stack()
/var/folders/j6/v7r4q64d1zv5qj54955b_k_00000gn/T/ipykernel_18606/2389047552.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  error_table.query("wMAPE != inf and mase != inf").groupby("model")[["mae","rmse","stability","wMAPE","mase"]].agg(['mean','median']).stack()


mae      rmse  stability     wMAPE      mase
model                                                                     
CrostonOptimized mean    4.442025  5.195877  17.987440  0.956469  1.097770
                 median  2.250000  2.684991   6.468486  0.555851  0.944273
HW52             mean    5.666718  6.593264  29.696056  1.115150  1.429038
                 median  2.754653  3.308298  13.580435  0.684064  1.080049
MA4              mean    4.390932  5.158533  27.983784  0.915407  1.027986
                 median  2.222222  2.633914  11.250000  0.578947  0.989130
Naive            mean    4.835846  5.661499  35.899181  0.978502  1.000000
                 median  2.500000  3.000000  16.000000  0.666667  1.000000
SN52             mean    6.031936  7.126418  35.743904  1.159529  1.566312
                 median  3.000000  3.674235  17.333334  0.777778  1.166667
StructuralTheta  mean    5.114506  5.950348  36.494694  0.955450  1.032169
                 median  2.512360  3.028923  15.079706  0.632937  1.000000